# Naive Neural Network

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Flatten, Dense
from sklearn.model_selection import train_test_split
import numpy as np
import keras

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow import keras



c:\Users\Georg\anaconda3\envs\steam_analysis\Lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.5 when it was built against 1.14.6, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [2]:
# Get current working directory 
folder_0 = os.path.join(os.getcwd(), 'resized_images', '0')
folder_1 = os.path.join(os.getcwd(), 'resized_images', '1')
folder_2 = os.path.join(os.getcwd(), 'resized_images', '2')
folder_3 = os.path.join(os.getcwd(), 'resized_images', '3')
folder_4 = os.path.join(os.getcwd(), 'resized_images', '4')
folder_5 = os.path.join(os.getcwd(), 'resized_images', '5')
folder_6 = os.path.join(os.getcwd(), 'resized_images', '6')
folder_7 = os.path.join(os.getcwd(), 'resized_images', '7')
folder_8 = os.path.join(os.getcwd(), 'resized_images', '8')
folder_9 = os.path.join(os.getcwd(), 'resized_images', '9')
folder_10 = os.path.join(os.getcwd(), 'resized_images', '10')
folder_11 = os.path.join(os.getcwd(), 'resized_images', '11')
folder_12 = os.path.join(os.getcwd(), 'resized_images', '12')
folder_13 = os.path.join(os.getcwd(), 'resized_images', '13')
folder_14 = os.path.join(os.getcwd(), 'resized_images', '14')
folder_15 = os.path.join(os.getcwd(), 'resized_images', '15')





In [3]:
def prepare_image_data(base_path='resized_images', num_classes=16):
    """
    Load images from folders 0-15 and prepare them with corresponding labels
    for neural network training.
    
    Args:
        base_path (str): Path to the directory containing class folders
        num_classes (int): Number of class folders (0 to num_classes-1)
    
    Returns:
        tuple: (images_array, labels_array) - NumPy arrays ready for neural network
    """
    images = []
    labels = []
    
    # Loop through each class folder
    for class_idx in range(num_classes):
        folder_path = os.path.join(os.getcwd(), base_path, str(class_idx))
        
        # Check if the folder exists
        if not os.path.exists(folder_path):
            print(f"Warning: Folder {folder_path} does not exist. Skipping.")
            continue
        
        # Get all image files in the folder
        image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
        
        print(f"Loading {len(image_files)} images from class {class_idx}")
        
        # Load each image
        for img_file in image_files:
            img_path = os.path.join(folder_path, img_file)
            try:
                # Read the image
                img = cv2.imread(img_path)
                
                # Check if image was loaded successfully
                if img is None:
                    print(f"Failed to load image: {img_path}")
                    continue
                
                # Add the image and its label to our lists
                images.append(img)
                labels.append(class_idx)
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
    
    # Convert lists to numpy arrays
    X = np.array(images)
    y = np.array(labels)
    
    print(f"Loaded {len(images)} images with shape {X.shape}")
    print(f"Labels shape: {y.shape}")
    
    return X, y



image_data, label_data = prepare_image_data()

Loading 12 images from class 0
Loading 12 images from class 1
Loading 12 images from class 2
Loading 12 images from class 3
Loading 12 images from class 4
Loading 12 images from class 5
Loading 12 images from class 6
Loading 12 images from class 7
Loading 12 images from class 8
Loading 12 images from class 9
Loading 12 images from class 10
Loading 12 images from class 11
Loading 12 images from class 12
Loading 12 images from class 13
Loading 12 images from class 14
Loading 12 images from class 15
Loaded 192 images with shape (192, 1000, 660, 3)
Labels shape: (192,)


In [4]:
# 1. Split the data
X_train, X_test, y_train, y_test = train_test_split(
    image_data, label_data, test_size=0.2, random_state=42
)

# 2. Normalize the pixel values (important for neural networks)
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# 3. Convert the labels to one-hot encoding
y_train_one_hot = keras.utils.to_categorical(y_train, num_classes=16)
y_test_one_hot = keras.utils.to_categorical(y_test, num_classes=16)

# 4. Create the model with correct input shape
model = keras.models.Sequential()
# Add a Conv2D layer to reduce the image size before flattening (optional but recommended)
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(1000, 660, 3)))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=512, activation='relu'))
model.add(Dense(units=16, activation='softmax'))

# 5. Compile the model correctly
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.categorical_crossentropy,  # Remove the parentheses
    metrics=['accuracy']  # Use string instead of keras.metrics.Accuracy()
)

# 6. Fit the model with one-hot encoded labels
# You might need to use a smaller batch size or downsample the images
# if you encounter memory issues
model.fit(
    X_train, 
    y_train_one_hot, 
    epochs=10, 
    batch_size=8,  # Smaller batch size due to large images
    validation_data=(X_test, y_test_one_hot)
)

c:\Users\Georg\anaconda3\envs\steam_analysis\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 110s 4s/step - accuracy: 0.0893 - loss: 65.3663 - val_accuracy: 0.0513 - val_loss: 2.7766
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 98s 4s/step - accuracy: 0.2551 - loss: 2.6807 - val_accuracy: 0.2051 - val_loss: 2.6303
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 151s 7s/step - accuracy: 0.5989 - loss: 1.4764 - val_accuracy: 0.4103 - val_loss: 2.2710
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 170s 8s/step - accuracy: 0.8500 - loss: 0.5666 - val_accuracy: 0.2564 - val_loss: 2.1831
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 113s 4s/step - accuracy: 0.9393 - loss: 0.2111 - val_accuracy: 0.4103 - val_loss: 2.1028
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 118s 5s/step - accuracy: 0.9816 - loss: 0.0546 - val_accuracy: 0.3590 - val_loss: 2.7323
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 125s 5s/step - accuracy: 0.9957 - loss: 0.0410 - val_accuracy: 0.4615 - val_loss: 1.6806
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 119s 5s/step - accuracy: 0.9933 - loss: 0.0261 - val_accuracy: 0.3846 - v

### Loss and Accuracy 

In [9]:
loss, accuracy = model.evaluate(X_test, y_test_one_hot, verbose=2)

2/2 - 15s - 7s/step - accuracy: 0.4359 - loss: 2.2878


### 